In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import time

In [3]:
df = pd.read_csv('X-IIoTID dataset_processed.csv')

In [10]:
X = (df.iloc[:,:-18]).drop(columns = ['class3'], axis=1)
# Y will be last 18 columns
y = df.iloc[:,-18:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# convert to numpy array
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = Sequential()
model.add(Conv1D(64, 3,activation="relu",input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(64, 3, activation="relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(128, 3, activation="relu"))
model.add(Conv1D(128, 3, activation="relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(18, activation="softmax"))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

mc = ModelCheckpoint('cnn_best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

start = time.time()
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[mc])
end = time.time()
print("Time taken to train the model: ", end-start)

Epoch 1/10
14901/14901 [==============================] - ETA: 0s - loss: 0.1157 - accuracy: 0.9663
Epoch 1: val_accuracy improved from -inf to 0.98013, saving model to cnn_best_model.h5
14901/14901 [==============================] - 276s 18ms/step - loss: 0.1157 - accuracy: 0.9663 - val_loss: 0.0557 - val_accuracy: 0.9801
Epoch 2/10
    1/14901 [..............................] - ETA: 4:38 - loss: 0.0476 - accuracy: 1.0000

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14898/14901 [============================>.] - ETA: 0s - loss: 0.0574 - accuracy: 0.9791
Epoch 2: val_accuracy improved from 0.98013 to 0.98475, saving model to cnn_best_model.h5
14901/14901 [==============================] - 268s 18ms/step - loss: 0.0574 - accuracy: 0.9791 - val_loss: 0.0403 - val_accuracy: 0.9847
Epoch 3/10
14899/14901 [============================>.] - ETA: 0s - loss: 0.0500 - accuracy: 0.9815
Epoch 3: val_accuracy did not improve from 0.98475
14901/14901 [==============================] - 301s 20ms/step - loss: 0.0500 - accuracy: 0.9815 - val_loss: 0.0414 - val_accuracy: 0.9844
Epoch 4/10
14901/14901 [==============================] - ETA: 0s - loss: 0.0470 - accuracy: 0.9823
Epoch 4: val_accuracy did not improve from 0.98475
14901/14901 [==============================] - 270s 18ms/step - loss: 0.0470 - accuracy: 0.9823 - val_loss: 0.0416 - val_accuracy: 0.9845
Epoch 5/10
14899/14901 [============================>.] - ETA: 0s - loss: 0.0453 - accuracy: 0.9833
Epoch

In [11]:
model = load_model('cnn_best_model.h5')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print("Time taken to predict: ", end-start)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

3726/3726 [==============================] - 17s 5ms/step
Time taken to predict:  17.897064208984375
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5168
           1       0.81      0.98      0.89       425
           2       1.00      0.99      1.00       299
           3       0.94      0.74      0.83      4011
           4       1.00      1.00      1.00      3280
           5       1.00      0.98      0.99       764
           6       1.00      1.00      1.00      6991
           7       1.00      1.00      1.00      3781
           8       1.00      0.18      0.30        17
           9       1.00      1.00      1.00       932
          10       0.98      1.00      0.99     67311
          11       1.00      1.00      1.00     15379
          12       0.96      0.96      0.96       167
          13       1.00      1.00      1.00      7633
          14       0.98      0.67      0.79       370
          15       0.00      0.00 

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [12]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Accuracy of each class:')
for i in range(cm.shape[0]):
    print(i, cm[i][i])

Accuracy of each class:
0 0.9998065015479877
1 0.9788235294117648
2 0.9933110367892977
3 0.7419596110695588
4 1.0
5 0.9764397905759162
6 0.9991417536833072
7 0.9984131182226924
8 0.17647058823529413
9 1.0
10 0.9950676709601699
11 0.9992197151960466
12 0.9640718562874252
13 0.9981658587711254
14 0.6675675675675675
15 0.0
16 0.7581395348837209
17 0.9995926680244399
